# 📈 Regression Analysis
## Modeling Relationships Between Variables

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-4-biometry/notebooks/08_regression_analysis.ipynb)

---

> *"All models are wrong, but some are useful."* - George E.P. Box

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand **simple linear regression** fundamentals
2. Interpret **slope, intercept, and R²**
3. Assess **regression assumptions** with diagnostics
4. Apply **multiple regression** with multiple predictors
5. Use **polynomial regression** for nonlinear relationships
6. Perform **model selection** and comparison
7. Make **predictions** with confidence intervals
8. Recognize when regression is **inappropriate**
9. Apply regression to **real ecological data**

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib scipy scikit-learn -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✅ Ready for regression analysis!")
print("📈 Let's model relationships in ecological data!")

---

## 📚 Part 1: What is Regression?

### Definition:

**Regression**: Statistical method for modeling the relationship between a **dependent variable (Y)** and one or more **independent variables (X)**.

### Goals of Regression:

1. **Describe relationships**: How does Y change with X?
2. **Make predictions**: Given new X, what Y do we expect?
3. **Test hypotheses**: Is there a significant relationship?
4. **Control for confounds**: What's X₁'s effect holding X₂ constant?

### Correlation vs Regression:

| Feature | Correlation | Regression |
|---------|-------------|------------|
| **Purpose** | Measure association | Model relationship |
| **Direction** | Symmetric (X↔Y) | Asymmetric (X→Y) |
| **Output** | r (correlation coefficient) | Equation Y = a + bX |
| **Prediction** | No | Yes |
| **Units** | Dimensionless | In Y units |

### Types of Regression:

#### **1. Simple Linear Regression**
```
Y = β₀ + β₁X + ε

One predictor (X), linear relationship
```

#### **2. Multiple Linear Regression**
```
Y = β₀ + β₁X₁ + β₂X₂ + ... + βₚXₚ + ε

Multiple predictors, linear relationships
```

#### **3. Polynomial Regression**
```
Y = β₀ + β₁X + β₂X² + β₃X³ + ... + ε

One predictor, nonlinear relationship
```

#### **4. Nonlinear Regression**
```
Y = f(X, β) + ε

Where f is nonlinear function (exponential, logistic, etc.)
```

### The Simple Linear Model:

```
Y = β₀ + β₁X + ε
```

Where:
- **Y** = Dependent variable (response)
- **X** = Independent variable (predictor)
- **β₀** = Intercept (Y when X = 0)
- **β₁** = Slope (change in Y per unit change in X)
- **ε** = Error (residual, unexplained variation)

### Fitting the Line:

**Method**: Ordinary Least Squares (OLS)

**Goal**: Minimize sum of squared residuals
```
SSE = Σ(Yᵢ - Ŷᵢ)²
```

Where:
- **Yᵢ** = Observed value
- **Ŷᵢ** = Predicted value (on the line)
- **Residual** = Yᵢ - Ŷᵢ (vertical distance to line)

### Visual Representation:

```
Y
↑
│     •  ← Residual (ε)
│    /|
│   / |
│  /  •
│ /  ← Regression line (Ŷ = β₀ + β₁X)
│/ •
│____________________→ X
     β₀ = intercept
```

---

## 📊 Part 2: Simple Linear Regression

### The Equation:

```
Y = β₀ + β₁X + ε
```

### Calculating Slope and Intercept:

**Slope (β₁)**:
```
β₁ = Σ[(Xᵢ - X̄)(Yᵢ - Ȳ)] / Σ[(Xᵢ - X̄)²]
   = Cov(X,Y) / Var(X)
   = r × (Sᵧ / Sₓ)
```

**Intercept (β₀)**:
```
β₀ = Ȳ - β₁X̄
```

### Interpreting the Slope:

**β₁ = 2.5**
- For every 1-unit increase in X
- Y increases by 2.5 units (on average)

**β₁ = -1.2**
- For every 1-unit increase in X
- Y decreases by 1.2 units (on average)

### Interpreting the Intercept:

**β₀ = 5**
- When X = 0
- Predicted Y = 5

**⚠️ Warning**: Intercept only meaningful if X = 0 is:
- Within the data range
- Biologically meaningful

### R² (Coefficient of Determination):

**Definition**: Proportion of variance in Y explained by X

**Formula**:
```
R² = 1 - (SSE / SST)
   = SSR / SST
```

Where:
- **SST** = Total sum of squares = Σ(Yᵢ - Ȳ)²
- **SSR** = Regression sum of squares = Σ(Ŷᵢ - Ȳ)²
- **SSE** = Error sum of squares = Σ(Yᵢ - Ŷᵢ)²

**Range**: 0 to 1
- **R² = 0**: X explains 0% of Y variation (no relationship)
- **R² = 0.5**: X explains 50% of Y variation
- **R² = 1**: X explains 100% of Y variation (perfect fit)

### Standard Error:

**Residual Standard Error (σ̂)**:
```
σ̂ = √[SSE / (n - 2)]
```

**Interpretation**: Average distance of points from the regression line

### Hypothesis Testing:

**Null Hypothesis**: H₀: β₁ = 0 (no relationship)

**Test Statistic**:
```
t = β₁ / SE(β₁)
```

**Degrees of freedom**: df = n - 2

**If p < 0.05**: Reject H₀, conclude significant relationship

In [ ]:
# Simple linear regression example: Plant height vs fertilizer amount
def simple_regression_example(n=50, seed=42):
    """
    Demonstrate simple linear regression with ecological data
    """
    np.random.seed(seed)
    
    # Generate data: Height increases with fertilizer
    fertilizer = np.random.uniform(0, 10, n)  # kg/hectare
    height = 20 + 3 * fertilizer + np.random.normal(0, 5, n)  # cm
    
    # Fit regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(fertilizer, height)
    r_squared = r_value ** 2
    
    # Predictions
    x_range = np.linspace(0, 10, 100)
    y_pred = intercept + slope * x_range
    
    # Calculate confidence interval (95%)
    # Simplified approach
    predict_ci = 1.96 * np.sqrt(np.var(height - (intercept + slope * fertilizer)))
    
    # Create visualization
    fig = go.Figure()
    
    # Scatter plot
    fig.add_trace(go.Scatter(
        x=fertilizer, y=height,
        mode='markers',
        marker=dict(size=10, color='lightblue', 
                   line=dict(width=1, color='darkblue')),
        name='Observed data',
        hovertemplate='Fertilizer: %{x:.1f} kg/ha<br>Height: %{y:.1f} cm<extra></extra>'
    ))
    
    # Regression line
    fig.add_trace(go.Scatter(
        x=x_range, y=y_pred,
        mode='lines',
        line=dict(width=3, color='red'),
        name=f'Regression line<br>Y = {intercept:.2f} + {slope:.2f}X'
    ))
    
    # Confidence bands
    fig.add_trace(go.Scatter(
        x=np.concatenate([x_range, x_range[::-1]]),
        y=np.concatenate([y_pred + predict_ci, (y_pred - predict_ci)[::-1]]),
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='95% Confidence band',
        hoverinfo='skip'
    ))
    
    # Add equation and stats
    fig.add_annotation(
        text=f"<b>Regression Equation:</b><br>" +
             f"Height = {intercept:.2f} + {slope:.2f} × Fertilizer<br><br>" +
             f"<b>Statistics:</b><br>" +
             f"R² = {r_squared:.3f}<br>" +
             f"p-value = {p_value:.4f}<br>" +
             f"n = {n}",
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        xanchor='left', yanchor='top',
        showarrow=False,
        bgcolor='lightyellow',
        bordercolor='black',
        borderwidth=1,
        font=dict(size=11)
    )
    
    fig.update_layout(
        title="📈 Simple Linear Regression: Plant Height vs Fertilizer<br><sub>Ordinary Least Squares (OLS) fit</sub>",
        xaxis_title="Fertilizer Amount (kg/hectare)",
        yaxis_title="Plant Height (cm)",
        height=600,
        template='plotly_white',
        hovermode='closest'
    )
    
    return fig, slope, intercept, r_squared, p_value, fertilizer, height

# Run example
fig, slope, intercept, r_squared, p_value, X, Y = simple_regression_example()
fig.show()

# Detailed interpretation
print("\n📈 Regression Analysis Results:\n")
print("   📐 EQUATION:")
print(f"      Height = {intercept:.2f} + {slope:.2f} × Fertilizer")
print("\n   📊 PARAMETER INTERPRETATION:")
print(f"      Intercept (β₀) = {intercept:.2f} cm")
print(f"         → Predicted height with NO fertilizer")
print(f"      Slope (β₁) = {slope:.2f} cm per kg/ha")
print(f"         → For every 1 kg/ha increase in fertilizer,")
print(f"         → height increases by {slope:.2f} cm on average")
print("\n   📈 MODEL FIT:")
print(f"      R² = {r_squared:.3f}")
print(f"         → Fertilizer explains {r_squared*100:.1f}% of height variation")
print(f"         → Remaining {(1-r_squared)*100:.1f}% due to other factors")
print("\n   🔬 SIGNIFICANCE TEST:")
print(f"      p-value = {p_value:.6f}")
if p_value < 0.001:
    print("         → HIGHLY significant (p < 0.001)")
elif p_value < 0.05:
    print("         → Significant (p < 0.05)")
else:
    print("         → Not significant (p ≥ 0.05)")
print("         → Strong evidence of relationship")
print("\n   💡 PRACTICAL MEANING:")
print("      • Adding fertilizer significantly increases plant height")
print(f"      • Every additional kg/ha yields ~{slope:.1f} cm more growth")
print(f"      • Model explains {r_squared*100:.0f}% of variation (good fit!)")
print("      • Can use this equation to predict height for new fertilizer amounts")
print("\n   📍 EXAMPLE PREDICTIONS:")
for fert in [2, 5, 8]:
    pred_height = intercept + slope * fert
    print(f"      Fertilizer = {fert} kg/ha  →  Predicted height = {pred_height:.1f} cm")

---

## 🔍 Part 3: Regression Assumptions & Diagnostics

### The Four Key Assumptions:

#### **1. Linearity**
```
The relationship between X and Y is linear

Check: Residuals vs Fitted plot should show no pattern
```

#### **2. Independence**
```
Observations are independent (no autocorrelation)

Check: Consider study design, plot residuals vs time/space
```

#### **3. Homoscedasticity** (Equal Variance)
```
Variance of residuals is constant across X values

Check: Residuals vs Fitted plot should show constant spread
```

#### **4. Normality**
```
Residuals are normally distributed

Check: Q-Q plot, histogram of residuals
```

### Diagnostic Plots:

#### **1. Residuals vs Fitted**
```
GOOD (assumptions met):
  •   •  •
• •  •  •  •
  • •  • •
────────────── (random scatter around zero)

BAD patterns:
  Curved: Nonlinearity (try polynomial)
  Funnel: Heteroscedasticity (try transformation)
  Clustered: Non-independence
```

#### **2. Q-Q Plot** (Quantile-Quantile)
```
GOOD:
  •
 •
•  ← Points fall on diagonal line
•
 •

BAD:
Tails deviate → Non-normal residuals
```

#### **3. Scale-Location**
```
Checks homoscedasticity
Should be flat horizontal line
```

#### **4. Residuals vs Leverage**
```
Identifies influential points
Cook's distance > 0.5 → Influential
```

### What to Do if Assumptions Violated:

**Nonlinearity**:
- Try polynomial regression
- Apply transformation (log, sqrt, etc.)
- Use nonlinear regression

**Heteroscedasticity**:
- Transform Y (log, sqrt)
- Use weighted least squares
- Use robust standard errors

**Non-normality**:
- Transform Y
- Use bootstrap
- Use non-parametric methods
- **Note**: With large n, less critical (CLT)

**Non-independence**:
- Use time series methods
- Mixed models for clustered data
- Account for spatial autocorrelation

In [ ]:
# Diagnostic plots for regression
def regression_diagnostics(X, Y):
    """
    Create diagnostic plots for regression assumptions
    """
    # Fit model
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, Y)
    
    # Calculate residuals and fitted values
    Y_fitted = intercept + slope * X
    residuals = Y - Y_fitted
    
    # Standardized residuals
    residuals_std = (residuals - np.mean(residuals)) / np.std(residuals)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Residuals vs Fitted (Check Linearity & Homoscedasticity)',
            'Q-Q Plot (Check Normality)',
            'Histogram of Residuals',
            'Residuals vs Order (Check Independence)'
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.12
    )
    
    # 1. Residuals vs Fitted
    fig.add_trace(
        go.Scatter(
            x=Y_fitted, y=residuals,
            mode='markers',
            marker=dict(size=8, color='blue'),
            showlegend=False
        ),
        row=1, col=1
    )
    # Add horizontal line at zero
    fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1)
    
    # 2. Q-Q Plot
    # Theoretical quantiles
    sorted_residuals = np.sort(residuals_std)
    n = len(sorted_residuals)
    theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, n))
    
    fig.add_trace(
        go.Scatter(
            x=theoretical_quantiles, y=sorted_residuals,
            mode='markers',
            marker=dict(size=8, color='green'),
            showlegend=False
        ),
        row=1, col=2
    )
    # Add diagonal reference line
    fig.add_trace(
        go.Scatter(
            x=[-3, 3], y=[-3, 3],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Histogram of residuals
    fig.add_trace(
        go.Histogram(
            x=residuals,
            nbinsx=15,
            marker_color='lightblue',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Residuals vs Order (time/sequence)
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(residuals)), y=residuals,
            mode='markers+lines',
            marker=dict(size=6, color='orange'),
            line=dict(width=1, color='orange'),
            showlegend=False
        ),
        row=2, col=2
    )
    fig.add_hline(y=0, line_dash="dash", line_color="red", row=2, col=2)
    
    # Update axes labels
    fig.update_xaxes(title_text="Fitted Values", row=1, col=1)
    fig.update_xaxes(title_text="Theoretical Quantiles", row=1, col=2)
    fig.update_xaxes(title_text="Residuals", row=2, col=1)
    fig.update_xaxes(title_text="Observation Order", row=2, col=2)
    
    fig.update_yaxes(title_text="Residuals", row=1, col=1)
    fig.update_yaxes(title_text="Standardized Residuals", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_yaxes(title_text="Residuals", row=2, col=2)
    
    fig.update_layout(
        title="🔍 Regression Diagnostic Plots<br><sub>Check assumptions visually</sub>",
        height=800,
        template='plotly_white'
    )
    
    return fig, residuals

# Run diagnostics on our data
fig_diag, residuals = regression_diagnostics(X, Y)
fig_diag.show()

# Statistical tests for assumptions
print("\n🔍 Regression Diagnostics:\n")
print("   ✅ ASSUMPTION CHECKS:")
print("\n   1️⃣ LINEARITY:")
print("      • Check 'Residuals vs Fitted' plot (top-left)")
print("      • Look for: Random scatter around zero")
print("      • Bad: Curved pattern, funnel shape")
print("\n   2️⃣ INDEPENDENCE:")
print("      • Check 'Residuals vs Order' plot (bottom-right)")
print("      • Look for: No pattern over time/sequence")
print("      • Bad: Trends, cycles, clusters")
print("\n   3️⃣ HOMOSCEDASTICITY (Equal Variance):")
print("      • Check 'Residuals vs Fitted' plot (top-left)")
print("      • Look for: Constant spread across X-axis")
print("      • Bad: Funnel shape (variance changes with X)")
print("\n   4️⃣ NORMALITY:")
print("      • Check 'Q-Q Plot' (top-right)")
print("      • Look for: Points fall on diagonal line")
print("      • Check 'Histogram' (bottom-left)")
print("      • Look for: Bell-shaped distribution")

# Shapiro-Wilk test for normality
statistic, p_value_sw = stats.shapiro(residuals)
print("\n   📊 FORMAL TESTS:")
print(f"      Shapiro-Wilk test (normality): p = {p_value_sw:.4f}")
if p_value_sw > 0.05:
    print("         → Residuals appear normally distributed (p > 0.05)")
else:
    print("         → Residuals may not be normal (p < 0.05)")
    print("         → Consider transformation or robust methods")

print("\n   💡 INTERPRETATION FOR OUR DATA:")
print("      • Plots look good - assumptions appear met")
print("      • Random scatter in residuals vs fitted")
print("      • Q-Q plot points follow diagonal line")
print("      • Histogram approximately bell-shaped")
print("      • No obvious pattern in order plot")
print("      → Regression results are trustworthy!")

---

## 📊 Part 4: Multiple Regression

### The Model:

```
Y = β₀ + β₁X₁ + β₂X₂ + ... + βₚXₚ + ε
```

### When to Use Multiple Regression:

1. **Multiple predictors** affect the response
2. **Control for confounds** (partial out effects)
3. **Test relative importance** of different variables
4. **Make better predictions** using multiple information

### Example: Plant Biomass

```
Biomass = β₀ + β₁(Rainfall) + β₂(Temperature) + β₃(Soil N) + ε
```

### Interpreting Coefficients:

**β₁ = 0.5** (Rainfall coefficient)
- Holding temperature and nitrogen constant
- 1 mm increase in rainfall
- Associated with 0.5 g/m² increase in biomass

**Key**: Each coefficient is the effect of that variable **holding all others constant**

### Adjusted R²:

**Problem**: R² always increases when adding variables
(Even if they're useless!)

**Solution**: Adjusted R²
```
R²ₐdⱼ = 1 - [(1 - R²)(n - 1) / (n - p - 1)]
```

- Penalizes for number of predictors (p)
- Only increases if new variable improves fit enough
- Use for model comparison

### Multicollinearity:

**Problem**: Predictors highly correlated with each other

**Example**:
```
Height = β₀ + β₁(Weight_kg) + β₂(Weight_lbs) + ε
```
**Weight in kg and lbs are perfectly correlated!**

**Symptoms**:
- Large coefficient standard errors
- Coefficients have wrong sign
- High R² but no significant predictors

**Check**: Variance Inflation Factor (VIF)
```
VIF > 10: Serious multicollinearity
VIF > 5:  Moderate concern
```

**Solutions**:
- Remove redundant variables
- Combine correlated variables (e.g., PCA)
- Collect more data
- Use ridge regression

In [ ]:
# Multiple regression example
def multiple_regression_example(n=100, seed=42):
    """
    Plant biomass predicted by rainfall, temperature, and soil nitrogen
    """
    np.random.seed(seed)
    
    # Generate predictors
    rainfall = np.random.uniform(300, 800, n)  # mm/year
    temperature = np.random.uniform(15, 30, n)  # °C
    soil_N = np.random.uniform(0, 50, n)  # kg/ha
    
    # Generate response with known coefficients
    # True model: Biomass = 50 + 0.3*Rain + 2*Temp + 1.5*N + noise
    biomass = (50 + 
               0.3 * rainfall + 
               2.0 * temperature + 
               1.5 * soil_N + 
               np.random.normal(0, 30, n))  # g/m²
    
    # Create DataFrame
    df = pd.DataFrame({
        'Rainfall': rainfall,
        'Temperature': temperature,
        'Soil_N': soil_N,
        'Biomass': biomass
    })
    
    # Fit multiple regression using sklearn
    X_multi = df[['Rainfall', 'Temperature', 'Soil_N']].values
    y = df['Biomass'].values
    
    model = LinearRegression()
    model.fit(X_multi, y)
    
    # Get coefficients
    intercept = model.intercept_
    coefs = model.coef_
    r2 = model.score(X_multi, y)
    
    # Adjusted R²
    n_samples = len(y)
    n_predictors = X_multi.shape[1]
    adj_r2 = 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_predictors - 1)
    
    # Create visualization: Actual vs Predicted
    y_pred = model.predict(X_multi)
    
    fig = go.Figure()
    
    # Scatter plot
    fig.add_trace(go.Scatter(
        x=y, y=y_pred,
        mode='markers',
        marker=dict(size=8, color='blue', opacity=0.6),
        name='Data points'
    ))
    
    # Perfect prediction line
    min_val = min(y.min(), y_pred.min())
    max_val = max(y.max(), y_pred.max())
    fig.add_trace(go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(color='red', dash='dash', width=2),
        name='Perfect prediction'
    ))
    
    # Add stats
    fig.add_annotation(
        text=f"<b>Multiple Regression Model:</b><br>" +
             f"Biomass = {intercept:.1f}<br>" +
             f"        + {coefs[0]:.3f} × Rainfall<br>" +
             f"        + {coefs[1]:.3f} × Temperature<br>" +
             f"        + {coefs[2]:.3f} × Soil N<br><br>" +
             f"<b>Model Fit:</b><br>" +
             f"R² = {r2:.3f}<br>" +
             f"Adjusted R² = {adj_r2:.3f}<br>" +
             f"n = {n}",
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        xanchor='left', yanchor='top',
        showarrow=False,
        bgcolor='lightyellow',
        bordercolor='black',
        font=dict(size=10)
    )
    
    fig.update_layout(
        title="📊 Multiple Regression: Biomass from 3 Predictors<br><sub>Actual vs Predicted values</sub>",
        xaxis_title="Actual Biomass (g/m²)",
        yaxis_title="Predicted Biomass (g/m²)",
        height=600,
        template='plotly_white'
    )
    
    return fig, df, model, coefs, intercept, r2, adj_r2

# Run example
fig_multi, df_multi, model_multi, coefs, intercept, r2, adj_r2 = multiple_regression_example()
fig_multi.show()

# Interpretation
print("\n📊 Multiple Regression Results:\n")
print("   📐 EQUATION:")
print(f"      Biomass = {intercept:.1f}")
print(f"              + {coefs[0]:.3f} × Rainfall")
print(f"              + {coefs[1]:.3f} × Temperature")
print(f"              + {coefs[2]:.3f} × Soil N")
print("\n   🔍 COEFFICIENT INTERPRETATION:")
print(f"\n      Rainfall ({coefs[0]:.3f}):")
print(f"         • Holding temperature & nitrogen constant")
print(f"         • 1 mm increase in rainfall")
print(f"         • → {coefs[0]:.3f} g/m² increase in biomass")
print(f"\n      Temperature ({coefs[1]:.3f}):")
print(f"         • Holding rainfall & nitrogen constant")
print(f"         • 1°C increase in temperature")
print(f"         • → {coefs[1]:.3f} g/m² increase in biomass")
print(f"\n      Soil Nitrogen ({coefs[2]:.3f}):")
print(f"         • Holding rainfall & temperature constant")
print(f"         • 1 kg/ha increase in soil N")
print(f"         • → {coefs[2]:.3f} g/m² increase in biomass")
print("\n   📈 MODEL FIT:")
print(f"      R² = {r2:.3f}")
print(f"         → Model explains {r2*100:.1f}% of biomass variation")
print(f"      Adjusted R² = {adj_r2:.3f}")
print(f"         → Accounts for number of predictors")
print("\n   💡 RELATIVE IMPORTANCE:")
print("      Temperature has strongest effect (largest coefficient)")
print("      All three predictors contribute meaningfully")
print("\n   🎯 PRACTICAL APPLICATION:")
print("      Can predict biomass for new sites given:")
print("      • Rainfall data")
print("      • Temperature data")
print("      • Soil nitrogen measurements")

---

## 📈 Part 5: Polynomial Regression

### When Relationships Aren't Linear:

Many ecological relationships are **nonlinear**:
- Growth curves (S-shaped)
- Optimum curves (hump-shaped)
- Saturation curves (asymptotic)

### Polynomial Regression:

**Quadratic (2nd degree)**:
```
Y = β₀ + β₁X + β₂X² + ε
```
- Creates parabola (U-shaped or ∩-shaped)
- One turning point
- Use for: Optimal temperature, light response

**Cubic (3rd degree)**:
```
Y = β₀ + β₁X + β₂X² + β₃X³ + ε
```
- Creates S-curve
- Two turning points
- Use for: Growth curves, dose-response

### Finding the Optimum:

For quadratic:
```
Optimal X = -β₁ / (2β₂)
```

### Dangers of High-Degree Polynomials:

**Overfitting**:
```
Degree 10 polynomial can fit 11 points perfectly
But predictions terrible!
```

**Rule**: Use lowest degree that captures pattern
- Usually: Quadratic (degree 2) or Cubic (degree 3)
- Rarely: Higher than degree 4

### Model Selection:

**Compare models using**:
- Adjusted R² (higher is better)
- AIC (lower is better)
- BIC (lower is better)
- Cross-validation error

**AIC (Akaike Information Criterion)**:
```
AIC = 2k - 2ln(L)
```
- k = number of parameters
- L = likelihood
- Lower AIC = better model
- Balances fit with complexity

In [ ]:
# Polynomial regression example: Photosynthesis vs Temperature
def polynomial_regression_example(n=50, seed=42):
    """
    Photosynthesis rate has optimal temperature (quadratic relationship)
    """
    np.random.seed(seed)
    
    # Generate data: Optimal at 25°C
    temperature = np.random.uniform(5, 45, n)
    # True quadratic relationship
    photosynthesis = (5 + 
                     3 * temperature - 
                     0.06 * temperature**2 + 
                     np.random.normal(0, 3, n))
    
    # Sort for plotting
    sort_idx = np.argsort(temperature)
    temp_sorted = temperature[sort_idx]
    photo_sorted = photosynthesis[sort_idx]
    
    # Fit models of different degrees
    degrees = [1, 2, 3]
    models = {}
    predictions = {}
    r2_scores = {}
    
    temp_range = np.linspace(5, 45, 200)
    
    for degree in degrees:
        # Create polynomial features
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(temperature.reshape(-1, 1))
        X_range_poly = poly.transform(temp_range.reshape(-1, 1))
        
        # Fit model
        model = LinearRegression()
        model.fit(X_poly, photosynthesis)
        
        # Predictions
        y_pred = model.predict(X_range_poly)
        
        # Store
        models[degree] = model
        predictions[degree] = y_pred
        r2_scores[degree] = r2_score(photosynthesis, model.predict(X_poly))
    
    # Create visualization
    fig = go.Figure()
    
    # Data points
    fig.add_trace(go.Scatter(
        x=temperature, y=photosynthesis,
        mode='markers',
        marker=dict(size=10, color='lightblue', 
                   line=dict(width=1, color='darkblue')),
        name='Observed data'
    ))
    
    # Fitted curves
    colors = {'1': 'red', '2': 'green', '3': 'purple'}
    names = {'1': 'Linear', '2': 'Quadratic', '3': 'Cubic'}
    
    for degree in degrees:
        fig.add_trace(go.Scatter(
            x=temp_range, y=predictions[degree],
            mode='lines',
            line=dict(width=3, color=colors[str(degree)]),
            name=f"{names[str(degree)]} (R²={r2_scores[degree]:.3f})"
        ))
    
    # Find and mark optimum for quadratic
    optimal_idx = np.argmax(predictions[2])
    optimal_temp = temp_range[optimal_idx]
    optimal_photo = predictions[2][optimal_idx]
    
    fig.add_trace(go.Scatter(
        x=[optimal_temp], y=[optimal_photo],
        mode='markers',
        marker=dict(size=15, color='gold', symbol='star',
                   line=dict(width=2, color='black')),
        name=f'Optimum: {optimal_temp:.1f}°C'
    ))
    
    fig.update_layout(
        title="📈 Polynomial Regression: Photosynthesis vs Temperature<br><sub>Quadratic captures optimal temperature</sub>",
        xaxis_title="Temperature (°C)",
        yaxis_title="Photosynthesis Rate (μmol CO₂/m²/s)",
        height=600,
        template='plotly_white'
    )
    
    return fig, models, r2_scores, optimal_temp, optimal_photo

# Run example
fig_poly, models_poly, r2_poly, opt_temp, opt_photo = polynomial_regression_example()
fig_poly.show()

# Interpretation
print("\n📈 Polynomial Regression Comparison:\n")
print("   📊 MODEL COMPARISON:")
for degree in [1, 2, 3]:
    model_name = ['Linear', 'Quadratic', 'Cubic'][degree-1]
    print(f"\n      {model_name} (Degree {degree}):")
    print(f"         R² = {r2_poly[degree]:.4f}")
    if degree == 1:
        print("         ❌ Misses the curved relationship")
        print("         Poor fit (underfit)")
    elif degree == 2:
        print("         ✅ Captures the optimal temperature")
        print("         Best choice (appropriate complexity)")
    else:
        print("         ⚠️ Slightly better R², but more complex")
        print("         Risk of overfitting")

print("\n   ⭐ OPTIMAL TEMPERATURE:")
print(f"      Temperature: {opt_temp:.1f}°C")
print(f"      Maximum photosynthesis: {opt_photo:.1f} μmol CO₂/m²/s")
print("\n   💡 ECOLOGICAL MEANING:")
print("      • Photosynthesis increases with temperature up to optimum")
print("      • Beyond optimum, enzyme denaturation causes decline")
print(f"      • Species adapted to ~{opt_temp:.0f}°C")
print("      • Climate change implications if temps exceed optimum!")
print("\n   🎯 MODEL SELECTION:")
print("      Choose QUADRATIC model because:")
      print("      1. Captures biological reality (optimal temperature)")
print("      2. Good R² without overfitting")
print("      3. Interpretable (can find optimum)")
print("      4. Parsimonious (simplest adequate model)")

---

## 🎓 Summary

### Key Concepts:

✅ **Regression**: Models relationship between Y and X(s)  
✅ **Simple Linear**: Y = β₀ + β₁X + ε  
✅ **Slope (β₁)**: Change in Y per unit change in X  
✅ **R²**: Proportion of variance explained (0 to 1)  
✅ **Assumptions**: Linearity, independence, homoscedasticity, normality  
✅ **Multiple**: Include multiple predictors simultaneously  
✅ **Polynomial**: Handle nonlinear relationships  
✅ **Diagnostics**: Check assumptions with residual plots  

### When to Use Regression:

**✅ Use regression when**:
- Predicting continuous Y from X
- Quantifying relationship strength
- Controlling for confounds
- Testing directional hypotheses

**❌ Don't use regression when**:
- Y is categorical (use logistic regression)
- Relationship clearly nonlinear (use appropriate model)
- Assumptions severely violated (transform or use robust methods)
- Sample size too small (n < 20 problematic)

### Regression vs ANOVA:

| Feature | Regression | ANOVA |
|---------|------------|-------|
| **X type** | Continuous | Categorical |
| **Y type** | Continuous | Continuous |
| **Question** | How does Y change with X? | Do groups differ in Y? |
| **Output** | Equation | Group means |
| **Prediction** | Yes | Limited |

**Note**: They're actually the same underlying model (General Linear Model)!

### Common Pitfalls:

❌ **Extrapolation**: Predicting beyond data range  
❌ **Overfitting**: Too many predictors for sample size  
❌ **Ignoring assumptions**: Using regression when violated  
❌ **Causation claims**: Correlation ≠ causation  
❌ **Data dredging**: Testing many X variables until something significant  
❌ **Ignoring outliers**: One point can strongly influence line  

### Best Practices:

**1. Plot First**: Always visualize data before fitting  
**2. Check Assumptions**: Run diagnostic plots  
**3. Keep It Simple**: Start with linear, add complexity if needed  
**4. Report Properly**: Include equation, R², p-value, n  
**5. Validate**: Test on new data if possible  
**6. Interpret Carefully**: Coefficients in context of study  

### Reporting Regression Results:

**Minimal** (for simple regression):
```
"Height increased significantly with fertilizer 
(β = 2.5 cm per kg/ha, R² = 0.68, p < 0.001, n = 50)"
```

**Complete** (in methods/results):
```
Methods: "We used simple linear regression to model 
plant height as a function of fertilizer amount."

Results: "Plant height increased significantly with 
fertilizer application (F₁,₄₈ = 102.5, p < 0.001). 
The regression equation was: Height = 20.3 + 2.5 × Fertilizer 
(R² = 0.68, n = 50). For every 1 kg/ha increase in fertilizer, 
height increased by 2.5 cm on average."
```

### Formulas Summary:

**Simple Linear**:
```
Y = β₀ + β₁X + ε
R² = 1 - (SSE / SST)
```

**Multiple**:
```
Y = β₀ + β₁X₁ + β₂X₂ + ... + βₚXₚ + ε
R²ₐdⱼ = 1 - [(1 - R²)(n - 1) / (n - p - 1)]
```

**Polynomial**:
```
Y = β₀ + β₁X + β₂X² + ... + βₚXᵖ + ε
```

---

<div align="center">

**Made with 💚 by Ms. Susama Kar & Dr. Alok Patel**

[📓 Previous: Experimental Design](07_experimental_design.ipynb) | 
[🏠 Unit 4 Home](../../)

</div>